<a href="https://colab.research.google.com/github/Anipro-10/Inverse-Design-of-Aging-Schedules-for-Al-Cu-Alloys-/blob/main/Al_Cu_Inverse_Design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('PRISMA Dataset.csv')

In [ ]:
df

,Temp,Time [d],Volume fraction of AL2CU_C16 (Bulk),Mean radius of AL2CU_C16 (Bulk) [m]
0,120,0.02733,7.799510e-19,2.645440e-10
1,120,0.02998,1.332160e-17,2.647080e-10
2,120,0.03097,3.869920e-17,2.648690e-10
3,120,0.03209,1.210340e-16,2.649280e-10
4,120,0.03329,3.787480e-16,2.649370e-10
...,...,...,...,...
13156,250,91.19386,4.417000e-02,3.952080e-08
13157,250,93.37177,4.417000e-02,3.982030e-08
13158,250,95.58998,4.417000e-02,4.013440e-08
13159,250,97.85037,4.417000e-02,4.044820e-08


In [ ]:
df = df.rename(columns = {
    'Temp' : 'T_age_C',
    'Time [d]' : 'time_days',
    'Volume fraction of AL2CU_C16 (Bulk)': 'vol_frac',
    'Mean radius of AL2CU_C16 (Bulk) [m]': 'r_mean_m'

})



In [ ]:
df

,T_age_C,time_days,vol_frac,r_mean_m
0,120,0.02733,7.799510e-19,2.645440e-10
1,120,0.02998,1.332160e-17,2.647080e-10
2,120,0.03097,3.869920e-17,2.648690e-10
3,120,0.03209,1.210340e-16,2.649280e-10
4,120,0.03329,3.787480e-16,2.649370e-10
...,...,...,...,...
13156,250,91.19386,4.417000e-02,3.952080e-08
13157,250,93.37177,4.417000e-02,3.982030e-08
13158,250,95.58998,4.417000e-02,4.013440e-08
13159,250,97.85037,4.417000e-02,4.044820e-08


In [ ]:
#Converting units

df['time_min'] = df['time_days']*24*60
df['r_mean_nm'] = df['r_mean_m']*1e9

In [ ]:
df

,T_age_C,time_days,vol_frac,r_mean_m,time_min,r_mean_nm
0,120,0.02733,7.799510e-19,2.645440e-10,39.3552,0.264544
1,120,0.02998,1.332160e-17,2.647080e-10,43.1712,0.264708
2,120,0.03097,3.869920e-17,2.648690e-10,44.5968,0.264869
3,120,0.03209,1.210340e-16,2.649280e-10,46.2096,0.264928
4,120,0.03329,3.787480e-16,2.649370e-10,47.9376,0.264937
...,...,...,...,...,...,...
13156,250,91.19386,4.417000e-02,3.952080e-08,131319.1584,39.520800
13157,250,93.37177,4.417000e-02,3.982030e-08,134455.3488,39.820300
13158,250,95.58998,4.417000e-02,4.013440e-08,137649.5712,40.134400
13159,250,97.85037,4.417000e-02,4.044820e-08,140904.5328,40.448200


In [ ]:
df = df.sample(frac = 1, random_state=0).reset_index(drop = True)

In [ ]:
df

,T_age_C,time_days,vol_frac,r_mean_m,time_min,r_mean_nm
0,220,1.88082,0.04490,5.235230e-09,2708.3808,5.235230
1,240,0.00514,0.04244,1.385840e-09,7.4016,1.385840
2,235,7.40477,0.04460,1.199320e-08,10662.8688,11.993200
3,205,0.08870,0.04436,1.416280e-09,127.7280,1.416280
4,245,40.68010,0.04433,2.686920e-08,58579.3440,26.869200
...,...,...,...,...,...,...
13156,250,41.00567,0.04414,3.027530e-08,59048.1648,30.275300
13157,165,0.00555,0.00230,3.360390e-10,7.9920,0.336039
13158,225,1.71758,0.04477,5.769630e-09,2473.3152,5.769630
13159,235,0.00151,0.04137,9.804620e-10,2.1744,0.980462


In [ ]:
def ys(r, v):
  sigma0 = 50 #base stress due to friction
  M = 3.0 #Taylor factor
  G = 27000.0 #Shear modulus of Al in MPa
  eps = 0.007 #Lattice misfit
  b = 0.286 #Burgers vector
  nu = 0.33 #Poisson's Ratio

  coherency_term = M * G * eps * np.sqrt(v)

  lambda_ = (2 * r) / (np.sqrt(3 * v))
  orowan_term = ((0.4 * G * b) / (2 * np.pi * np.sqrt(1-nu) * lambda_)) * np.log(r/b)

  ys = sigma0 + coherency_term + orowan_term
  return ys

In [ ]:
df['Yield Stress'] = ys(df['r_mean_nm'], df['vol_frac'])


In [ ]:
df

,T_age_C,time_days,vol_frac,r_mean_m,time_min,r_mean_nm,Yield Stress
0,220,1.88082,0.04490,5.235230e-09,2708.3808,5.235230,231.346583
1,240,0.00514,0.04244,1.385840e-09,7.4016,1.385840,288.819910
2,235,7.40477,0.04460,1.199320e-08,10662.8688,11.993200,203.961078
3,205,0.08870,0.04436,1.416280e-09,127.7280,1.416280,293.161799
4,245,40.68010,0.04433,2.686920e-08,58579.3440,26.869200,187.894748
...,...,...,...,...,...,...,...
13156,250,41.00567,0.04414,3.027530e-08,59048.1648,30.275300,185.951195
13157,165,0.00555,0.00230,3.360390e-10,7.9920,0.336039,89.160812
13158,225,1.71758,0.04477,5.769630e-09,2473.3152,5.769630,227.277389
13159,235,0.00151,0.04137,9.804620e-10,2.1744,0.980462,298.259959


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
X = df[['T_age_C', 'time_min']].values
y = df['Yield Stress'].values

In [ ]:
df_test = pd.read_csv('test_Al_Cu.csv')

In [ ]:
df_test = df_test.rename(columns = {
    'Temp' : 'T_age_C',
    'Time [d]' : 'time_days',
    'Volume fraction of AL2CU_C16 (Bulk)': 'vol_frac',
    'Mean radius of AL2CU_C16 (Bulk) [m]': 'r_mean_m'

})


In [ ]:
#Converting units

df_test['time_min'] = df_test['time_days']*24*60
df_test['r_mean_nm'] = df_test['r_mean_m']*1e9

In [ ]:
df_test = df_test.sample(frac = 1, random_state=0).reset_index(drop = True)

In [ ]:
df_test

,T_age_C,time_days,vol_frac,r_mean_m,time_min,r_mean_nm
0,110,0.16984,7.963030e-10,2.595470e-10,244.5696,0.259547
1,157,0.02339,3.571000e-02,5.149940e-10,33.6816,0.514994
2,157,0.69645,4.518000e-02,8.450510e-10,1002.8880,0.845051
3,213,0.00684,4.276000e-02,9.372630e-10,9.8496,0.937263
4,213,0.18151,4.456000e-02,2.081230e-09,261.3744,2.081230
...,...,...,...,...,...,...
1228,213,0.00060,3.537000e-02,5.962160e-10,0.8640,0.596216
1229,213,0.00421,4.230000e-02,8.558540e-10,6.0624,0.855854
1230,213,69.17571,4.532000e-02,1.437910e-08,99613.0224,14.379100
1231,157,7.69498,4.588000e-02,1.413200e-09,11080.7712,1.413200


In [ ]:
df_test.shape

(1233, 6)

In [ ]:
df_test['Yield Stress'] = ys(df_test['r_mean_nm'], df_test['vol_frac'])

In [ ]:
df_test

,T_age_C,time_days,vol_frac,r_mean_m,time_min,r_mean_nm,Yield Stress
0,110,0.16984,7.963030e-10,2.595470e-10,244.5696,0.259547,50.010512
1,157,0.02339,3.571000e-02,5.149940e-10,33.6816,0.514994,269.398458
2,157,0.69645,4.518000e-02,8.450510e-10,1002.8880,0.845051,312.256511
3,213,0.00684,4.276000e-02,9.372630e-10,9.8496,0.937263,303.454552
4,213,0.18151,4.456000e-02,2.081230e-09,261.3744,2.081230,274.391506
...,...,...,...,...,...,...,...
1228,213,0.00060,3.537000e-02,5.962160e-10,0.8640,0.596216,277.159496
1229,213,0.00421,4.230000e-02,8.558540e-10,6.0624,0.855854,303.616712
1230,213,69.17571,4.532000e-02,1.437910e-08,99613.0224,14.379100,200.872599
1231,157,7.69498,4.588000e-02,1.413200e-09,11080.7712,1.413200,297.395335


In [ ]:
X_test = df_test[['T_age_C', 'time_min']].values
y_test = df_test['Yield Stress'].values

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
#Feature Engineering

In [ ]:
df['log_time'] = np.log10(df['time_min'])
df['T_log_time'] = df['T_age_C'] * df['log_time']

df_test['log_time'] = np.log10(df_test['time_min'])
df_test['T_log_time'] = df_test['T_age_C'] * df_test['log_time']

In [ ]:
X = df[['T_age_C' ,'log_time', 'T_log_time']].values
y = df['Yield Stress'].values

X_test = df_test[['T_age_C', 'log_time', 'T_log_time']].values
y_test = df_test['Yield Stress'].values

In [ ]:
rf_feat = RandomForestRegressor(n_estimators=50, random_state=0)
rf_feat.fit(X, y)

RandomForestRegressor(n_estimators=50, random_state=0)

In [ ]:
y_pred = rf_feat.predict(X_test)
r2 = r2_score(y_test, y_pred)
print(f"R² score on test set: {r2:.4f}")

R² score on test set: 0.9467


In [ ]:
T_max, T_min = df['T_age_C'].max(), df['T_age_C'].min()

In [ ]:
logt_max, logt_min = df['log_time'].max(), df['log_time'].min()

In [ ]:
def cost(T, logt):
  t = np.exp(logt)
  norm_T = (T - T_min) / (T_max - T_min)
  norm_t = (t - np.exp(logt_min)) / (np.exp(logt_max) - np.exp(logt_min))

  return 0.3*norm_T + 2*norm_t


In [ ]:
def recommend_aging(YS_target, tol = 1.0):

  T_grid = np.arange(T_min, T_max+0.5, 0.5)
  logt_grid = np.arange(logt_min, logt_max, 0.005)

  TT, LL = np.meshgrid(T_grid, logt_grid)
  candidates = np.column_stack([TT.ravel(), LL.ravel()])

  T_vals = candidates[:, 0]
  logt_vals = candidates[:, 1]

  T_log = T_vals * logt_vals

  X_cand = np.column_stack([T_vals, logt_vals, T_log])

  ys_pred = rf_feat.predict(X_cand)

  mask = np.abs(ys_pred - YS_target) <= tol
  feasible = candidates[mask]

  if feasible.shape[0] == 0:
    raise ValueError(f"No (T, log_t) within ±{tol} MPa of {YS_target} MPa")

  costs = cost(feasible[:, 0], feasible[:, 1])

  best_idx = np.argmin(costs)
  T_best, logt_best = feasible[best_idx]
  t_best = np.exp(logt_best)

  return T_best, t_best

In [ ]:
target = 140

T_optimal, t_optimal = recommend_aging(target, tol = 1)
print(f"Optimal schedule for {target} MPa: T = {T_optimal:.1f}°C, t = {t_optimal:.1f} min")

Optimal schedule for 230 MPa: T = 153.0°C, t = 4.6 min


In [ ]:
sigma_orowan

array([3.74756369e-14, 1.75908580e-13, 2.98946660e-13, 5.28938802e-13,
       9.35809613e-13, 1.65618114e-12, 2.93100732e-12, 5.18648022e-12,
       9.17667011e-12, 1.62342924e-11, 2.87147655e-11, 5.07833790e-11,
       8.97989089e-11, 1.58761151e-10, 2.79450347e-10, 4.65288952e-10,
       7.38796844e-10, 8.33253096e-10])